- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [22]:
REQUEST = '''{
    "args": {
        "date": ["2013-02-27"],
        "start_date": ["2013-01-27"],
        "end_date": ["2013-02-27"],
        "lat": ["52"],
        "lon": ["-1"]
    },
    "path": {
        "parameter": "temperature",
        "operation": "min"
    }
}'''

In [2]:
# GET /helloworld

print('hello world')


hello world


In [3]:
import datetime
import iris

iris.FUTURE.netcdf_promote = True

def load_data(pattern):
    # pattern is glob style
    cubes = iris.load('/opt/data/{}'.format(pattern))
    return iris.cube.CubeList(cubes).concatenate_cube()


In [4]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

In [5]:

def get_coords(cube, lat, lon):
    x, y = transform_coords(cube, lon, lat)
    first_year = next(cube.slices_over('time'))
    samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
    first_point = first_year.interpolate(samples, iris.analysis.Nearest())
    exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
    exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
    
    return(exact_x, exact_y)
    


In [6]:

def collapse_latlon(cube, x, y):
    x_const = iris.Constraint(projection_x_coordinate=x)
    y_const = iris.Constraint(projection_y_coordinate=y)
    return cube.extract(x_const).extract(y_const)

In [7]:
from datetime import timedelta

def expand_years(start, end, past_years=10):
    years = [
        datetime.datetime(
            d.year - i,
            d.month,
            d.day)
        for i in range(past_years)
        for d in select_all_days(start, end)]
    return years

def select_all_days(start, end):
    delta = end - start
    days = [end - timedelta(days=i) for i in range(delta.days)]
    return days

def extract_dates(cube, dates):
    time_units = cube.coord('time').units
    vals = [time_units.date2num(date) for date in dates]
    time_constraint = iris.Constraint(time=vals)
    return cube.extract(time_constraint)

In [15]:
def mean(cube):
    return cube.collapsed('time', iris.analysis.MEAN)

def cmax(cube):
    return cube.collapsed('time', iris.analysis.MAX)

def cmin(cube):
    return cube.collapsed('time', iris.analysis.MIN)

In [16]:
patterns = {
    'temperature': 'maximum-temperature/*.nc',
    'rainfall': 'rainfall/*.nc'
}

operations = {
    'mean': mean,
    'max': cmax,
    'min': cmin
}

In [25]:
from collections import defaultdict
CACHED_POINTS = defaultdict(lambda: None)

In [23]:
def get_response(cube, dates, x, y, operation):
    time_cube = collapse_latlon(cube, x, y)
    collapsed_time_cube = extract_dates(time_cube, dates)
    
    val = float(operation(collapsed_time_cube).data)
    
    response = {
        'value': val,
        'start_date': dates[-1].strftime('%Y-%m-%d'),
        'end_date': dates[0].strftime('%Y-%m-%d')
    }
    return response

In [33]:
def parse_query(req):
    query = req['args']
    lat = float(args['lat'][0])
    lon = float(args['lon'][0])
    start, end = parse_date_range(query)
    return lat, lon, start, end

def parse_path(req):
    path = req['path']
    param = path['parameter']
    op = path['operation']
    return param, op

def parse_date_range(query):
    if 'start_date' and 'end_date' in query.keys():
        start_date = datetime.datetime.strptime(args['start_date'][0], '%Y-%m-%d')
        end_date = datetime.datetime.strptime(args['end_date'][0], '%Y-%m-%d')
    elif 'date' in query.keys():
        start_date = datetime.datetime.strptime(args['date'][0], '%Y-%m-%d')
        end_date = start_date + datetime.timedelta(days=1)
    else:
        start_date = datetime.datetime.now()
        end_date = start_date + datetime.timedelta(days=1)
    return start_date, end_date

In [43]:
# GET /:parameter/:operation/climatology
import json

req = json.loads(REQUEST)

param, op = parse_path(req)
lat, lon, start, end = parse_query(req)

data_path = patterns[param]
operation = operations[op]

cube = load_data(data_path)
x, y = get_coords(cube, lat, lon)

cache_key = ('climatology', param, x, y, start, end, operation)

if CACHED_POINTS[cache_key] != None:
    response = CACHED_POINTS[cache_key]
else:
    dates = expand_years(start_date)
    response = get_response(cube, dates, x, y, operation)
    CACHED_POINTS[cache_key] = response

print(json.dumps(response))

{"value": 1.8729329109191895, "start_date": "2004-01-27", "end_date": "2013-01-27"}


In [42]:
# GET /:parameter/:operation/range
import json

req = json.loads(REQUEST)

param, op = parse_path(req)
lat, lon, start, end = parse_query(req)

data_path = patterns[param]
operation = operations[op]

cube = load_data(data_path)
x, y = get_coords(cube, lat, lon)

cache_key = ('range', param, x, y, start, end, operation)

if CACHED_POINTS[cache_key] != None:
    response = CACHED_POINTS[cache_key]
else:
    dates = select_all_days(start, end)
    response = get_response(cube, dates, x, y, operation)
    CACHED_POINTS[cache_key] = response

print(json.dumps(response))

{"value": 0.7139625549316406, "start_date": "2013-01-28", "end_date": "2013-02-27"}


In [57]:
# ResponseInfo GET /:parameter/mean/range
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"headers": {"Content-Type": "application/json"}}


In [ ]:
# ResponseInfo GET /:parameter/mean/climatology
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))